In [3]:
# Auto Load module
%load_ext autoreload
%autoreload 2
from CohortPCA import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
ControlPanel = "data/AncestryPCA.master.panel"
EVEC = "/Users/jiayao/Work/EsophagealAtresia/PCA/CARE.hg19.gz.plus.HapMap.pca.evec"

instance = CohortPCA(ControlPanel, EVEC)
instance.loadControlPanel()
instance.loadEigenValue()
#instance.ReduceSuperPop()
#instance.Plot("RGN")
#instance.MarkerWithKNN()
instance.MarkerWithSVM()

Model: RandomForestClassifier
Fitting Model...
Done with Fitting, used 0.1017 secs.
CrossV: [ 0.97089041  0.97770154  0.97938144  0.97758621  0.98096886]
Accuracy: 0.97731 (+/- 0.00688)


In [12]:
def modifyPop(pop):
    pop = pop.strip()
    if pop in ["African American","Black","black"]:
        return "AFR"
    if pop in ["Caucasian","White","caucasian"]:
        return "EUR"
    if pop in ["Asian","Asian (Bangladesh)","Asian (Bangladeshi)","Asian (Korean)","Asian (Philippines)"]:
        return "SAS"
    else:
        return pop

# Annotate Ped 
selfR = {}
pcaR = {}

reader = csv.reader(open("/Users/jiayao/Work/EsophagealAtresia/ALL.ped", 'rb'), delimiter="\t")
header = reader.next()
H = header
for row in reader:
    row[7] = modifyPop(row[7])
    selfR[row[1]]=row[7]
    
reader = csv.reader(open("CaseAnnotationSVM.txt", 'rb'), delimiter="\t")
for row in reader:
    pcaR[row[0]]=row[1]

H = H[0:6]
H.extend(["pcaR.PoP"])
Ped = csv.reader(open("/Users/jiayao/Work/EsophagealAtresia/ALL.ped", 'rb'), delimiter="\t")
Ped.next()
Out = csv.writer(open("/Users/jiayao/Work/EsophagealAtresia/ALL.pop.ped", 'wb'), delimiter="\t")
Out.writerow(H)
for row in Ped:
    row.append(pcaR.get(row[1], '.'))
    Out.writerow(row)



In [19]:
df = pd.read_csv("/Users/jiayao/Work/EsophagealAtresia/ALL.pop.ped", sep="\t")
df = df[df["Relationship"] == "Proband"]
df.groupby('pcaR.PoP').count()["ProbandID"]



pcaR.PoP
AFR     2
AJ      7
AMR     1
DOMI    2
EUR     7
SAS     2
Name: ProbandID, dtype: int64

In [53]:
df.groupby('selfR.PoP').count()["SAMPLEID"]

selfR.PoP
AFR                            9
Asian (Indian)                 4
Black/Hispanic                 1
Caucasian/African American     1
Caucasian/Asian (India)        1
Caucasian/Hispanic             5
EUR                           59
Hispanic                      31
Hispanic/Asian/Caucasian       1
Mixed (Indian/Hispanic)        1
Other                          1
SAS                           10
Name: SAMPLEID, dtype: int64

In [65]:
for item in list(df[df["pcaR.PoP"].isin(["."])]["SAMPLEID"]):
    print item

d274
